In [1]:
# %pip install -q openai anthropic peft sentencepiece protobuf

In [2]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'

In [3]:
import os
import json
from tqdm.auto import tqdm

In [4]:
import sys
sys.path.append('/home/majed_alshaibani/Projects/jrcai_corekit/src')

In [5]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) # suppress requests warnings of InsecureRequestWarning

In [6]:
from llm import *


/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [7]:
os.environ["OPENAI_API_KEY"] = "sk-SLCxlbRQUx70uDw0-4FWaRyCdzh1MpbJgqyLhpK-azT3BlbkFJO-jSkK6IOK1lvShwmUt9DkA6BSpetmL9jjxRtHrvIA"

In [8]:
messsage_generator = MessageGeneratorFromOpenAIAPI()

# Exploring Jais outputs

In [9]:
def get_chat_completion(messages):
    llm_response = messsage_generator(messages)
    return llm_response[0]

test the API

In [10]:
print(get_chat_completion(
    messages=[[{
            'role': 'user',
            'content': 'ما هو لون نجوم السماء؟'
        }]]
    )
)

لون نجوم السماء يمكن أن يتراوح بين عدة ألوان استنادًا إلى درجة حرارة النجم وعناصره الكيميائية. عمومًا، يمكن تلخيص ألوان النجوم كالتالي:

1. **النجوم الزرقاء**: هذه النجوم هي الأكثر حرارة، ودرجات حرارتها تكون أعلى من 10,000 كلفن.

2. **النجوم البيضاء**: تقع هذه النجوم ضمن درجة حرارة متوسطة، بين 7,500 و10,000 كلفن.

3. **النجوم الصفراء**: مثل شمسنا، التي تبلغ درجة حرارتها حوالي 5,500 كلفن.

4. **النجوم البرتقالية**: درجة حرارتها أقل، تتراوح بين 3,500 و5,000 كلفن.

5. **النجوم الحمراء**: هي الأبرد بين النجوم، وتكون درجات حرارتها أقل من 3,500 كلفن.

التباين في الألوان يعود إلى الاختلافات في درجة حرارة النجم والعناصر الكيميائية التي يتكون منها الضوء الصادر عنه.


# Exploring the dataset

In [11]:
import datasets

In [12]:
dataset = datasets.load_dataset('arbml/AraSum')
dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'summary', 'article'],
        num_rows: 49603
    })
})

In [13]:
articles = dataset['train']['article']
summaries = dataset['train']['summary']
len(articles), len(summaries)

(49603, 49603)

In [14]:
# max words, min words, avg words for summaries
max_summaries_words = max(len(text.split()) for text in summaries)
min_summaries_words = min(len(text.split()) for text in summaries)
avg_summaries_words = sum(len(text.split()) for text in summaries) / len(summaries)
max_summaries_words, min_summaries_words, avg_summaries_words

(45, 2, 33.53730621131786)

In [15]:
# max words, min words, avg words for articles
max_articles_words = max(len(text.split()) for text in articles)
min_articles_words = min(len(text.split()) for text in articles)
avg_articles_words = sum(len(text.split()) for text in articles) / len(articles)
max_articles_words, min_articles_words, avg_articles_words

(2898, 33, 376.54426143580025)

In [16]:
considered_articles,considered_summaries = [],[]
for article,summary in zip(articles,summaries):
    if 100 < len(article.split()) < 500 and 20 < len(summary.split()) < 50:
        considered_articles.append(article)
        considered_summaries.append(summary)
len(considered_articles),len(considered_summaries)

(36873, 36873)

In [17]:
articles = considered_articles[:3000]
summaries = considered_summaries[:3000]
len(articles),len(summaries)

(3000, 3000)

# Generating the dataset

first, we generate articles by polishing them.

In [18]:
def save_articles_to_jsonl(articles, file_path):  
    # Append new articles
    with open(file_path, 'w', encoding='utf-8') as f:
        for article in articles:
            json_object = article
            f.write(json.dumps(json_object, ensure_ascii=False) + '\n')

def load_articles_from_jsonl(file_path):
    articles = []
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                articles.append(json.loads(line))
    return articles

In [19]:
# Your existing code to generate articles
articles_by_polishing_prompts = []

for article in articles:
    prompt = f"""
قم بإعادة صياغة هذه المقالة التالية، مراعيا تدقيقها لغويا ونحويا وبنائيا ومعنى، المقالة كتبت لتنشر في الصحافة.
قم بإعادة الصياغة فقط دون إضافة أي تعبيرات أخرى قبل إعادة الصياغة أو بعدها
المقالة:
{article}
    """.strip()
    articles_by_polishing_prompts.append(prompt)


# Main process
!mkdir -p generated_arabic_datasets/openai/arasum
file_path = "generated_arabic_datasets/openai/arasum/generated_articles_from_polishing.jsonl"
generated_articles_from_polishing = load_articles_from_jsonl(file_path)

for i, prompt in tqdm(
    enumerate(
        articles_by_polishing_prompts[len(generated_articles_from_polishing) :],
        start=len(generated_articles_from_polishing),
    ),
    total=len(articles),
    initial=len(generated_articles_from_polishing)
):
    output_article = {}
    generated = get_chat_completion(
        [[
            {
                "role": "user",
                "content": prompt,
            },
        ]]
    )
    output_article["original_article"] = articles[i]
    output_article["original_article_summary"] = summaries[i]
    output_article["generated_article"] = generated
    generated_articles_from_polishing.append(output_article)
    # Save the generated articles
    save_articles_to_jsonl(generated_articles_from_polishing, file_path)

  0%|          | 0/3000 [00:00<?, ?it/s]